In [1]:
import pandas as pd
import dwsemear
import datetime
import collections
import numpy as np

import postgresemear as sql  # mover os arquivos para a pasta principal

def calcula_idade(dados, coluna):
    idade = []
    for i in dados.index:
        if dwsemear.e_nulo(dados.loc[i,coluna]):
            idade.append(None)
        else:
            anos = (dados.loc[i,'datahora'] - dados.loc[i,coluna]).days
            anos = int(round(anos/365))
            idade.append(anos)
    dados['idade'] = idade
    return dados

def remove_localize(data):
    return data.tz_localize(None)

def coluna_para_lista(coluna):
    nova_coluna = []
    for elem in coluna:
        if dwsemear.e_nulo(elem):
            nova_coluna.append(None)
        else:
            nova_coluna.append(elem)
    return nova_coluna

def para_int(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    else:
        return int(elemento)

def para_float(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    else:
        return float(elemento)

def para_str(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    else:
        return str(elemento)
    
def para_data(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    else:
        return elemento
    
def pegue_elementos_com_repeticao(coluna):
    contagem = collections.Counter(coluna)
    return list(getItemsUpToCount(contagem,1).keys())

def getItemsUpToCount(dct, n):
    upToCount = {}
    for key in dct:
        if dct[key] > n:
            upToCount[key] = dct[key]
    return upToCount


In [44]:
COLUNAS_PDD = ['ID',
            'DataDeReferencia',
            'DataDeOriginiacao',
            'DataDaProximaParcela',
            'DataDeVencimento',
            'DataDeRenegociacao',
            'CPF',
            'CodigoDoProduto',
            'codigo_contrato',
            'DiasEmAtraso',
            'QuantidadeDeParcelasAbertas',
            'ValorSaldo',
            'ValorPDD',
            'ValorOperacao',
            'RendaDoCliente',
            'ValorDaProximaParcela',
            'RatingContabilFim']


COLUNAS =  [
            'ID',
            'codigo_proposta',
            'data_originacao',
            'uf',
            'genero',
            'renda_cliente',
            'score',
            'valor_parcela',
            'renda_presumida_unitfour',
            'atraso_medio',
            'ultimo_pedido',
            'idade',
            'cpf',
            'quantidade_parcelas',
            'associados_distintos_30_dias',
            'quantidade_consultas',
            'codigo_contrato',
            'codigo_proposta_crivo',
            'dias_atraso']

In [3]:
dados_abt = sql.query_para_pandas('''SELECT *
                      FROM crivo, producao
                      WHERE crivo.ultimo_pedido = 1
                      AND crivo.codigo_proposta = producao.codigo_proposta_crivo
                      ''')

In [4]:
dados_abt.columns = COLUNAS
dados_abt = dados_abt.drop(['ID','codigo_proposta_crivo', 'ultimo_pedido'],axis=1)
dados_abt.data_originacao = dados_abt.data_originacao.apply(remove_localize)
dados_abt['inadimplente'] = dados_abt.dias_atraso.apply(lambda x: 1 if x > 60 else 0)

In [21]:
abt_2018_1 = dados_abt[(dados_abt.data_originacao >= datetime.datetime(2018,1,1)) &
          (dados_abt.data_originacao <= datetime.datetime(2018,6,30))]
abt_2018_2 = dados_abt[(dados_abt.data_originacao >= datetime.datetime(2018,7,1)) &
          (dados_abt.data_originacao <= datetime.datetime(2018,12,31))]
abt_2019_1 = dados_abt[(dados_abt.data_originacao >= datetime.datetime(2019,1,1)) &
          (dados_abt.data_originacao <= datetime.datetime(2019,6,30))]

In [27]:
abt_2018_1.to_csv('190903_ABT_2018_1.csv',sep=';', index = False)
abt_2018_2.to_csv('190903_ABT_2018_2.csv',sep=';', index = False)
abt_2019_1.to_csv('190903_ABT_2019_1.csv',sep=';', index = False)

# ENRIQUECIMENTO COM PDD

In [2]:
dados_abt = sql.query_para_pandas('''SELECT *
                      FROM crivo, producao
                      WHERE crivo.ultimo_pedido = 1
                      AND crivo.codigo_proposta = producao.codigo_proposta_crivo
                      ''')
dados_abt.columns = COLUNAS
dados_abt = dados_abt.drop(['ID','codigo_proposta_crivo', 'ultimo_pedido'],axis=1)

In [25]:
dados_abt.data_originacao = dados_abt.data_originacao.apply(remove_localize)

In [40]:
parametros = (datetime.datetime(2019,1,31),)
dados_pdd = sql.query_para_pandas('''SELECT *
                      FROM pdd
                      WHERE DataDeReferencia = %s
                      ''',parametros)
dados_pdd.columns = COLUNAS_PDD
joinDf = dados_abt.set_index('codigo_contrato').join(dados_pdd.set_index('codigo_contrato'))
dados_abt['jan-19'] = joinDf.DiasEmAtraso.tolist()

In [85]:
parametros = (datetime.datetime(2019,2,28),)
dados_pdd = sql.query_para_pandas('''SELECT *
                      FROM pdd
                      WHERE DataDeReferencia = %s
                      ''',parametros)
dados_pdd.columns = COLUNAS_PDD
joinDf = dados_abt.set_index('codigo_contrato').join(dados_pdd.set_index('codigo_contrato'))
dados_abt['fev-19'] = joinDf.DiasEmAtraso.tolist()

In [86]:
parametros = (datetime.datetime(2019,3,31),)
dados_pdd = sql.query_para_pandas('''SELECT *
                      FROM pdd
                      WHERE DataDeReferencia = %s
                      ''',parametros)
dados_pdd.columns = COLUNAS_PDD
joinDf = dados_abt.set_index('codigo_contrato').join(dados_pdd.set_index('codigo_contrato'))
dados_abt['mar-19'] = joinDf.DiasEmAtraso.tolist()

In [87]:
parametros = (datetime.datetime(2019,4,30),)
dados_pdd = sql.query_para_pandas('''SELECT *
                      FROM pdd
                      WHERE DataDeReferencia = %s
                      ''',parametros)
dados_pdd.columns = COLUNAS_PDD
joinDf = dados_abt.set_index('codigo_contrato').join(dados_pdd.set_index('codigo_contrato'))
dados_abt['abr-19'] = joinDf.DiasEmAtraso.tolist()

In [88]:
parametros = (datetime.datetime(2019,5,31),)
dados_pdd = sql.query_para_pandas('''SELECT *
                      FROM pdd
                      WHERE DataDeReferencia = %s
                      ''',parametros)
dados_pdd.columns = COLUNAS_PDD
joinDf = dados_abt.set_index('codigo_contrato').join(dados_pdd.set_index('codigo_contrato'))
dados_abt['mai-19'] = joinDf.DiasEmAtraso.tolist()

In [89]:
parametros = (datetime.datetime(2019,6,30),)
dados_pdd = sql.query_para_pandas('''SELECT *
                      FROM pdd
                      WHERE DataDeReferencia = %s
                      ''',parametros)
dados_pdd.columns = COLUNAS_PDD
joinDf = dados_abt.set_index('codigo_contrato').join(dados_pdd.set_index('codigo_contrato'))
dados_abt['jun-19'] = joinDf.DiasEmAtraso.tolist()